Cartpole v0 prueba

In [1]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 10000 # Number of times the enviroment is ran
LOSS_CLIPPING = 0.2 # Approximated values stated in the original paper
ENTROPY_LOSS = 0.001 # Epochs to train the network (recommended between 3 and 30)
LR = 0.0003 # Learning rate 
NUM_ACTIONS = 2 # Number of possible action in the environment
NUM_STATES = 4 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 64 # Batch size for the neural nets
BUFFER_SIZE = 2048 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
LAMBDA = 0.95 # Used in the original paper un the GAE
NORMALIZE = True # Whether to normalize GAE or not



# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Softmax as there are different probabilities depending on the action
        output = Dense(NUM_ACTIONS, activation="softmax")(X)
        
        # Compile the model with the custom loss
        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss=self.ppo_loss, optimizer=OPTIMIZER(lr=LR))

    # Custom loss functions for the PPO
    def ppo_loss(self, y_true, y_pred):
        # Unpack the elements given in the true label
        advantages, true_label, actions = y_true[:, :1], y_true[:, 1:1+NUM_ACTIONS], y_true[:, 1+NUM_ACTIONS:]
        
        prob = actions * y_pred
        old_prob = actions * true_label 

        ratio = K.exp(K.log(prob + 1e-10) - K.log(old_prob + 1e-10))
        
        p1 = ratio * advantages
        p2 = K.clip(ratio, min_value=1 - LOSS_CLIPPING, max_value=1 + LOSS_CLIPPING) * advantages

        actor_loss = -K.mean(K.minimum(p1, p2))

        entropy = -(y_pred * K.log(y_pred + 1e-10))
        entropy = ENTROPY_LOSS * K.mean(entropy)
        
        total_loss = actor_loss - entropy

        return total_loss

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Linear output to know how good the action is
        value = Dense(1)(X)
        
        # Compile it with mse loss and gradient descent
        self.model = Model(inputs=X_input, outputs = value)
        self.model.compile(loss='mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_PPO_Actor.h5"
        self.Critic_name = f"{self.env_name}_PPO_Critic.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.model.predict(state)[0]
        
        # Probability based to choose the action
        action = np.random.choice(NUM_ACTIONS, p=prediction)
        action_onehot = np.zeros([NUM_ACTIONS])
        action_onehot[action] = 1
        return action, action_onehot, prediction

    # Generalized Advantage Estimation implemented in the original paper
    def get_gaes(self, rewards, dones, values, next_values):
        # Dones are used to track when is the final step of an episode, so next values are no applied
        deltas = [r + GAMMA * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        
        # Convert list to array as .mean() and .std() are used later
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * GAMMA * LAMBDA * gaes[t + 1]

        target = gaes + values
        if NORMALIZE:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Get the advantage
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))

        # Stack info to unpack it in the custom loss
        y_true = np.hstack([advantages, predictions, actions])

        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, target, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)

 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, action_onehot, prediction = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action_onehot)
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction)
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            LR *= 0.95
                            K.set_value(self.Actor.model.optimizer.learning_rate, LR)
                            K.set_value(self.Critic.model.optimizer.learning_rate, LR)
                            
                        if average > 200:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.episode >= EPISODES:
                break
        self.env.close()  
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'CartPole-v0'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score:   20.00		average:   20.00
Episode:     2		score:   54.00		average:   37.00
Episode:     3		score:   20.00		average:   31.33
Episode:     4		score:   23.00		average:   29.25
Episode:     5		score:   27.00		average:   28.80
Episode:     6		score:   21.00		average:   27.50
Episode:     7		score:   17.00		average:   26.00
Episode:     8		score:   23.00		average:   25.62
Episode:     9		score:   34.00		average:   26.56
Episode:    10		score:   12.00		average:   25.10
Episode:    11		score:   24.00		average:   25.00
Episode:    12		score:   29.00		average:   25.33
Episode:    13		score:   21.00		average:   25.00
Episode:    14		score:   26.00		average:   25.07
Episode:    15		score:   14.00		average:   24.33
Episode:    16		score:   27.00		average:   24.50
Episode:    17		score:   26.00		average:   24.59
Episode:    18		score:   28.00		average:   24.78
Episode:    19		score:   12.00		avera

Episode:   160		score:   89.00		average:   33.01
Episode:   161		score:  138.00		average:   34.11
Episode:   162		score:   37.00		average:   34.37
Episode:   163		score:   47.00		average:   34.68
Episode:   164		score:   43.00		average:   34.92
Episode:   165		score:   47.00		average:   34.95
Episode:   166		score:   43.00		average:   35.17
Episode:   167		score:   70.00		average:   35.57
Episode:   168		score:   21.00		average:   35.56
Episode:   169		score:   93.00		average:   36.22
Episode:   170		score:   40.00		average:   36.39
Episode:   171		score:   27.00		average:   36.53
Episode:   172		score:   50.00		average:   36.41
Episode:   173		score:   30.00		average:   36.55
Episode:   174		score:   27.00		average:   36.72
Episode:   175		score:   67.00		average:   37.16
Episode:   176		score:   30.00		average:   37.35
Episode:   177		score:   36.00		average:   37.51
Episode:   178		score:   96.00		average:   38.31
Episode:   179		score:   74.00		average:   38.49
Episode:   180		scor

Episode:   329		score:   57.00		average:   82.35
Episode:   330		score:   40.00		average:   82.37
Episode:   331		score:   45.00		average:   82.08
Episode:   332		score:  142.00		average:   82.95
Episode:   333		score:   44.00		average:   82.91
Episode:   334		score:   66.00		average:   82.86
Episode:   335		score:   89.00		average:   81.75
Episode:   336		score:   53.00		average:   81.77
Episode:   337		score:   11.00		average:   80.67
Episode:   338		score:   72.00		average:   81.02
Episode:   339		score:   49.00		average:   81.21
Episode:   340		score:   62.00		average:   80.32
Episode:   341		score:   76.00		average:   80.38
Episode:   342		score:  107.00		average:   81.10
Episode:   343		score:   59.00		average:   81.08
Episode:   344		score:   78.00		average:   80.70
Episode:   345		score:  200.00		average:   81.87
Episode:   346		score:  200.00		average:   83.63
Episode:   347		score:   55.00		average:   83.85
Episode:   348		score:   41.00		average:   82.83
Episode:   349		scor

Episode:   497		score:   74.00		average:   83.92
Episode:   498		score:   61.00		average:   83.11
Episode:   499		score:   84.00		average:   82.65
Episode:   500		score:   40.00		average:   82.70
Episode:   501		score:  165.00		average:   83.63
Episode:   502		score:   54.00		average:   83.53
Episode:   503		score:  200.00		average:   85.08
Episode:   504		score:   68.00		average:   84.12
Episode:   505		score:   51.00		average:   84.14
Episode:   506		score:  129.00		average:   85.08
Episode:   507		score:   40.00		average:   84.23
Episode:   508		score:   79.00		average:   84.49
Episode:   509		score:   69.00		average:   84.30
Episode:   510		score:   32.00		average:   84.21
Episode:   511		score:  127.00		average:   85.15
Episode:   512		score:  160.00		average:   85.65
Episode:   513		score:  114.00		average:   86.28
Episode:   514		score:  107.00		average:   86.34
Episode:   515		score:   62.00		average:   85.85
Episode:   516		score:   42.00		average:   85.78
Episode:   517		scor

Episode:   665		score:   44.00		average:   90.14
Episode:   666		score:  200.00		average:   91.14
Episode:   667		score:   63.00		average:   91.39
Episode:   668		score:   42.00		average:   90.11
Episode:   669		score:  133.00		average:   91.13
Episode:   670		score:   98.00		average:   90.78
Episode:   671		score:  154.00		average:   91.56
Episode:   672		score:  156.00		average:   92.03
Episode:   673		score:   85.00		average:   92.58
Episode:   674		score:   72.00		average:   92.97
Episode:   675		score:   68.00		average:   91.92
Episode:   676		score:  110.00		average:   91.34
Episode:   677		score:  112.00		average:   91.04
Episode:   678		score:   33.00		average:   90.98
Episode:   679		score:   84.00		average:   90.86
Episode:   680		score:   59.00		average:   90.50
Episode:   681		score:   78.00		average:   90.81
Episode:   682		score:  152.00		average:   91.40
Episode:   683		score:  119.00		average:   91.90
Episode:   684		score:  104.00		average:   91.56
Episode:   685		scor

...

Episode:  3026		score:  165.00		average:   89.54
Episode:  3027		score:  117.00		average:   90.25
Episode:  3028		score:  200.00		average:   91.90
Episode:  3029		score:   81.00		average:   91.39
Episode:  3030		score:  200.00		average:   92.21
Episode:  3031		score:   65.00		average:   90.86
Episode:  3032		score:   36.00		average:   90.82
Episode:  3033		score:  148.00		average:   91.42
Episode:  3034		score:   72.00		average:   91.80
Episode:  3035		score:  111.00		average:   92.51
Episode:  3036		score:   37.00		average:   90.88
Episode:  3037		score:   50.00		average:   89.87
Episode:  3038		score:   86.00		average:   88.80
Episode:  3039		score:  162.00		average:   90.30
Episode:  3040		score:   37.00		average:   90.29
Episode:  3041		score:   20.00		average:   89.22
Episode:  3042		score:   67.00		average:   88.35
Episode:  3043		score:  142.00		average:   89.21
Episode:  3044		score:  139.00		average:   89.29
Episode:  3045		score:   34.00		average:   89.10
Episode:  3046		scor

Episode:  3194		score:  200.00		average:   79.76
Episode:  3195		score:  177.00		average:   80.73
Episode:  3196		score:   45.00		average:   79.65
Episode:  3197		score:   30.00		average:   79.59
Episode:  3198		score:   87.00		average:   80.01
Episode:  3199		score:   94.00		average:   80.35
Episode:  3200		score:   53.00		average:   80.66
Episode:  3201		score:   95.00		average:   81.25
Episode:  3202		score:  194.00		average:   81.46
Episode:  3203		score:  197.00		average:   82.40
Episode:  3204		score:   35.00		average:   80.95
Episode:  3205		score:   27.00		average:   80.24
Episode:  3206		score:   87.00		average:   80.54
Episode:  3207		score:   98.00		average:   81.06
Episode:  3208		score:  118.00		average:   81.92
Episode:  3209		score:  123.00		average:   81.78
Episode:  3210		score:   53.00		average:   81.01
Episode:  3211		score:  156.00		average:   81.75
Episode:  3212		score:   67.00		average:   81.40
Episode:  3213		score:   73.00		average:   81.79
Episode:  3214		scor

Episode:  3361		score:   74.00		average:   96.88
Episode:  3362		score:  158.00		average:   98.07
Episode:  3363		score:   71.00		average:   97.02
Episode:  3364		score:  131.00		average:   97.45
Episode:  3365		score:   86.00		average:   97.53
Episode:  3366		score:   35.00		average:   97.61
Episode:  3367		score:   49.00		average:   97.24
Episode:  3368		score:   77.00		average:   97.55
Episode:  3369		score:  154.00		average:   98.47
Episode:  3370		score:  126.00		average:   99.17
Episode:  3371		score:  156.00		average:   99.22
Episode:  3372		score:   39.00		average:   98.32
Episode:  3373		score:   74.00		average:   97.29
Episode:  3374		score:  177.00		average:   98.89
Episode:  3375		score:   19.00		average:   97.71
Episode:  3376		score:  124.00		average:   97.59
Episode:  3377		score:   34.00		average:   96.44
Episode:  3378		score:   35.00		average:   95.63
Episode:  3379		score:   45.00		average:   94.56
Episode:  3380		score:  200.00		average:   95.43
Episode:  3381		scor

Episode:  3528		score:   48.00		average:  103.24
Episode:  3529		score:   30.00		average:  102.20
Episode:  3530		score:   60.00		average:  102.03
Episode:  3531		score:   81.00		average:  101.29
Episode:  3532		score:   36.00		average:  100.40
Episode:  3533		score:   44.00		average:  100.54
Episode:  3534		score:  123.00		average:  101.33
Episode:  3535		score:   40.00		average:  100.91
Episode:  3536		score:   96.00		average:  101.18
Episode:  3537		score:   41.00		average:  100.32
Episode:  3538		score:  200.00		average:  101.57
Episode:  3539		score:   24.00		average:  101.46
Episode:  3540		score:   39.00		average:  101.18
Episode:  3541		score:  188.00		average:  102.29
Episode:  3542		score:   92.00		average:  101.80
Episode:  3543		score:  140.00		average:  101.47
Episode:  3544		score:  113.00		average:  101.35
Episode:  3545		score:  106.00		average:  101.88
Episode:  3546		score:   80.00		average:  102.12
Episode:  3547		score:  200.00		average:  103.33
Episode:  3548		scor

KeyboardInterrupt: 